In [24]:
!pip install pygeohash

  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6152 sha256=98de3632f513678df81dbd80c071fef5b5c4791f74643186e7dca52ea4753890
  Stored in directory: /home/student/.cache/pip/wheels/70/98/c5/332f0986813a345d8869d98d134e5c89e322399d5450b1b05b
Successfully built pygeohash


In [1]:
import json
import pygeohash as pgh
from pymongo import MongoClient
from pyspark.sql import SparkSession,Row
import pprint
import ast
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'

In [2]:
hostip = "192.168.0.100" # change it to your IP

spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('Assignment 3 Part B Streaming Application')
    .getOrCreate()
)

spark

In [3]:
kafka_sdf = (
    spark.readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', f'{hostip}:9092')
    .option('subscribe', "stream_1")
    .load()
)


In [4]:
kafka_sdf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
hotspot = kafka_sdf.select('value') 

In [40]:
client = MongoClient('192.168.0.100', 27017) 
db = client.fit3182_assignment_db
db.fit3182_partb.drop() # drop everything in the collection to have new data insert to the collection

In [50]:
def process_batch(df, epoch_id):
    # Transform and write batchDF
    temp = df.collect()

    data = []
    for i in temp:
        data.append(ast.literal_eval(i.value.decode("utf-8")))
        
    # Get Geohash3 encodings for all three streams
    geohash3 = ""
    for d in data:
        lat = d['latitude']
        lon = d['longitude']
        geohash3 = pgh.encode(lat, lon, precision=3)
        
    # if climate stream has any matching hotspot (Aqua or Terra) hotspot
    hotspot_array = []
    geo_temp = ""
    for d in data:
        if d['producer'] == 'climate':
            geo_temp = geohash3
            break

    for d in data:
        if d['producer'] == 'hotspot' and geohash3 == geo_temp:
            hotspot_array.append(d)
            
    for d in data:
        if d['producer'] == 'climate':
            # if no, just store climate data in mongo and end
            if len(hotspot_array) == 0:
                collection = db.fit3182_partb
                collection.insert_one(d)
            else:
                d['hotspots_data'] = hotspot_array
   
    # else, there must be one or more matching hotspot records
    # compute geohash5 for each hotspot data
    geohash5 = ""
    for d in data:
        geohash5 = pgh.encode(lat, lon, precision=5)
    
    # get the geohash5 for each climate and see if there is any matching geohash5
    geo5_temp = ""
    for d in data:
        if d['producer'] == 'climate':
            geo5_temp = geohash5
    
    # compute geohash5 for each hotspot data
    avg_s_temp = []
    avg_c_temp = []
    fire_data = []
    for d in data:
        # two different hotspot from two different satellite only can find the average between
        if d['producer'] == 'hotspot' and geohash5 == geo5_temp and d['satellite'] == "AQUA":
            avg_s_temp.append(d['surface_temperature_celcius'])
            avg_c_temp.append(d['confidence'])
            for f in data:
                if d['producer'] == 'hotspot' and geohash5 == geo5_temp and d['satellite'] != "AQUA":
                    avg_s_temp.append(d['surface_temperature_celcius'])
                    avg_c_temp.append(d['confidence'])
        # same but if TERRA first
        elif d['producer'] == 'hotspot' and geohash5 == geo5_temp and d['satellite'] != "AQUA":
            avg_s_temp.append(d['surface_temperature_celcius'])
            avg_c_temp.append(d['confidence'])
            for f in data:
                if d['producer'] == 'hotspot' and geohash5 == geo5_temp and d['satellite'] == "AQUA":
                    avg_s_temp.append(d['surface_temperature_celcius'])
                    avg_c_temp.append(d['confidence'])
    
    # group up and compute average of confidence and surface temperature for hotspots with same geohash5
    if len(avg_s_temp) != 0 and len(avg_c_temp) != 0:
        s = sum(avg_s_temp)/len(avg_s_temp)
        c = sum(avg_c_temp)/len(avg_c_temp)
        temp = {'average_surface_temperature_celcius':s,'average_confidence':c} # save it as a dict
        fire_data.append(temp)
        if d['producer'] == 'hotspot':
            d['fire_data'] = fire_data

    # determine if fire is natural (air_temperature > 20, GHI > 100)
    for d in data:
        if d['producer'] == "climate":
            if d['air_temperature_celcius'] > 20 and d['GHI_w/m2'] > 100:
                d['fire_status'] = "natural"
            else:
                d['fire_status'] = "other"
            collection = db.fit3182_partb
            collection.insert_one(d)
            print(d)


In [51]:
writer = (
    hotspot.writeStream.format("Console")
    .outputMode('append')
    .trigger(processingTime='10 seconds') # every 10 seconds will trigger a batch
    .foreachBatch(process_batch)
)

In [52]:
try:
    query = writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopping query.')
finally:
    query.stop()

{'latitude': -37.1926, 'longitude': 143.8095, 'air_temperature_celcius': 8.0, 'relative_humidity': 36.3, 'windspeed_knots': 6.3, 'max_wind_speed': 13.0, 'precipitation ': ' 0.01G', 'GHI_w/m2': 76.0, '_id': 1972, 'producer': 'climate', 'created_date': '28/10/2028 07:59:19', 'hotspots_data': [{'latitude': -38.4412, 'longitude': 146.3188, 'confidence': 74.0, 'surface_temperature_celcius': 48.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '21/10/2028 22:23:22'}, {'latitude': -36.5985, 'longitude': 143.1836, 'confidence': 100.0, 'surface_temperature_celcius': 103.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '22/10/2028 03:11:21'}, {'latitude': -37.5632, 'longitude': 142.2864, 'confidence': 69.0, 'surface_temperature_celcius': 45.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '22/10/2028 03:11:22'}, {'latitude': -36.4142, 'longitude': 143.1077, 'confidence': 100.0, 'surface_temperature_celcius': 97.0, 'producer': 'hotspot', 'satellite': 'AQ

{'latitude': -36.1002, 'longitude': 142.3405, 'air_temperature_celcius': 23.0, 'relative_humidity': 49.6, 'windspeed_knots': 6.3, 'max_wind_speed': 13.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 198.0, '_id': 1976, 'producer': 'climate', 'created_date': '01/11/2028 07:59:19', 'hotspots_data': [{'latitude': -37.6403, 'longitude': 142.8696, 'confidence': 74.0, 'surface_temperature_celcius': 48.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '25/10/2028 12:47:22'}, {'latitude': -37.6403, 'longitude': 142.8696, 'confidence': 74.0, 'surface_temperature_celcius': 48.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '25/10/2028 17:35:21'}, {'latitude': -36.2455, 'longitude': 141.3205, 'confidence': 62.0, 'surface_temperature_celcius': 43.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '25/10/2028 17:35:22'}, {'latitude': -35.4255, 'longitude': 143.6306, 'confidence': 89.0, 'surface_temperature_celcius': 65.0, 'producer': 'hotspot', 'satellite': 'AQU

{'latitude': -37.596, 'longitude': 149.319, 'air_temperature_celcius': 19.0, 'relative_humidity': 56.1, 'windspeed_knots': 6.9, 'max_wind_speed': 12.0, 'precipitation ': ' 0.00A', 'GHI_w/m2': 155.0, '_id': 1980, 'producer': 'climate', 'created_date': '05/11/2028 07:59:19', 'hotspots_data': [{'latitude': -37.6261, 'longitude': 142.9447, 'confidence': 85.0, 'surface_temperature_celcius': 59.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '29/10/2028 12:47:22'}, {'latitude': -37.5915, 'longitude': 143.1328, 'confidence': 81.0, 'surface_temperature_celcius': 54.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '29/10/2028 17:35:21'}, {'latitude': -36.8763, 'longitude': 142.7266, 'confidence': 68.0, 'surface_temperature_celcius': 44.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '29/10/2028 17:35:22'}, {'latitude': -37.0167, 'longitude': 143.8586, 'confidence': 87.0, 'surface_temperature_celcius': 62.0, 'producer': 'hotspot', 'satellite': 'AQUA'

{'latitude': -37.0046, 'longitude': 141.8047, 'air_temperature_celcius': 10.0, 'relative_humidity': 41.4, 'windspeed_knots': 6.9, 'max_wind_speed': 8.9, 'precipitation ': ' 0.00G', 'GHI_w/m2': 92.0, '_id': 1984, 'producer': 'climate', 'created_date': '09/11/2028 07:59:19', 'hotspots_data': [{'latitude': -35.4616, 'longitude': 143.7251, 'confidence': 80.0, 'surface_temperature_celcius': 53.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '02/11/2028 12:47:22'}, {'latitude': -37.959, 'longitude': 143.5497, 'confidence': 60.0, 'surface_temperature_celcius': 40.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '02/11/2028 17:35:21'}, {'latitude': -37.9085, 'longitude': 141.1038, 'confidence': 98.0, 'surface_temperature_celcius': 82.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '02/11/2028 17:35:22'}, {'latitude': -37.7752, 'longitude': 141.9086, 'confidence': 88.0, 'surface_temperature_celcius': 64.0, 'producer': 'hotspot', 'satellite': 'AQUA',

{'latitude': -37.453, 'longitude': 148.118, 'air_temperature_celcius': 6.0, 'relative_humidity': 39.0, 'windspeed_knots': 1.8, 'max_wind_speed': 5.1, 'precipitation ': ' 0.01G', 'GHI_w/m2': 56.0, '_id': 1988, 'producer': 'climate', 'created_date': '13/11/2028 07:59:19', 'hotspots_data': [{'latitude': -37.38, 'longitude': 149.334, 'confidence': 61.0, 'surface_temperature_celcius': 31.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '06/11/2028 12:47:22'}, {'latitude': -36.304, 'longitude': 143.7445, 'confidence': 59.0, 'surface_temperature_celcius': 40.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '06/11/2028 17:35:21'}, {'latitude': -37.396, 'longitude': 148.086, 'confidence': 100.0, 'surface_temperature_celcius': 65.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '06/11/2028 17:35:22'}, {'latitude': -37.2109, 'longitude': 142.7966, 'confidence': 74.0, 'surface_temperature_celcius': 52.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'creat

...

{'latitude': -36.752, 'longitude': 144.15, 'air_temperature_celcius': 14.0, 'relative_humidity': 50.4, 'windspeed_knots': 12.8, 'max_wind_speed': 20.0, 'precipitation ': ' 0.01G', 'GHI_w/m2': 120.0, '_id': 2312, 'producer': 'climate', 'created_date': '03/10/2029 07:59:19', 'hotspots_data': [{'latitude': -37.7805, 'longitude': 143.5003, 'confidence': 95.0, 'surface_temperature_celcius': 75.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '26/09/2029 03:11:22'}, {'latitude': -37.4224, 'longitude': 147.0511, 'confidence': 100.0, 'surface_temperature_celcius': 113.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '26/09/2029 07:59:21'}, {'latitude': -36.111, 'longitude': 145.124, 'confidence': 85.0, 'surface_temperature_celcius': 66.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '26/09/2029 07:59:22'}, {'latitude': -37.6572, 'longitude': 142.0703, 'confidence': 97.0, 'surface_temperature_celcius': 80.0, 'producer': 'hotspot', 'satellite': 'AQUA'

{'latitude': -36.6851, 'longitude': 141.6125, 'air_temperature_celcius': 17.0, 'relative_humidity': 59.0, 'windspeed_knots': 9.4, 'max_wind_speed': 15.0, 'precipitation ': ' 0.75G', 'GHI_w/m2': 135.0, '_id': 2316, 'producer': 'climate', 'created_date': '07/10/2029 07:59:19', 'hotspots_data': [{'latitude': -38.1536, 'longitude': 144.1275, 'confidence': 77.0, 'surface_temperature_celcius': 50.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '30/09/2029 03:11:22'}, {'latitude': -37.3333, 'longitude': 143.8079, 'confidence': 75.0, 'surface_temperature_celcius': 48.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '30/09/2029 07:59:21'}, {'latitude': -36.9303, 'longitude': 143.1034, 'confidence': 70.0, 'surface_temperature_celcius': 72.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '30/09/2029 07:59:22'}, {'latitude': -36.7128, 'longitude': 141.3986, 'confidence': 86.0, 'surface_temperature_celcius': 60.0, 'producer': 'hotspot', 'satellite': 'AQU

{'latitude': -37.538, 'longitude': 148.895, 'air_temperature_celcius': 6.0, 'relative_humidity': 37.1, 'windspeed_knots': 5.8, 'max_wind_speed': 9.9, 'precipitation ': ' 0.00I', 'GHI_w/m2': 57.0, '_id': 2320, 'producer': 'climate', 'created_date': '11/10/2029 07:59:19', 'hotspots_data': [{'latitude': -36.6188, 'longitude': 144.0402, 'confidence': 69.0, 'surface_temperature_celcius': 44.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '04/10/2029 03:11:22'}, {'latitude': -36.7315, 'longitude': 143.9232, 'confidence': 67.0, 'surface_temperature_celcius': 44.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'created_date': '04/10/2029 07:59:21'}, {'latitude': -35.723, 'longitude': 142.063, 'confidence': 64.0, 'surface_temperature_celcius': 54.0, 'producer': 'hotspot', 'satellite': 'TERRA', 'created_date': '04/10/2029 07:59:22'}, {'latitude': -36.7501, 'longitude': 142.8867, 'confidence': 56.0, 'surface_temperature_celcius': 49.0, 'producer': 'hotspot', 'satellite': 'AQUA', 'cr

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopping query.
